In [1]:
import pandas as pd

# Charger le dataset avec le bon nom de fichier
try:
    df = pd.read_csv("NBA_Data.csv")
    print("Dataset chargé avec succès !")
    print(f"Dimensions : {df.shape}")
    print("Colonnes :", list(df.columns))
    print("-" * 30)
    print(df.head())
except FileNotFoundError:
    print("Erreur : Le fichier 'NBA_Data.csv' est introuvable.")

Dataset chargé avec succès !
Dimensions : (9130, 24)
Colonnes : ['player', 'team', 'POS', 'GP', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'REB', 'AST', 'STL', 'BLK', 'TO', 'DD2', 'TD3', 'year', 'salary']
------------------------------
                    player team POS  GP   MIN   PTS   FGM   FGA   FG%  3PM  \
0              Luka Doncic  DAL  PG  70  37.5  33.9  11.5  23.6  48.7  4.1   
1    Giannis Antetokounmpo  MIL  PF  73  35.2  30.4  11.5  18.8  61.1  0.5   
2  Shai Gilgeous-Alexander  OKC  PG  75  34.0  30.1  10.6  19.8  53.5  1.3   
3            Jalen Brunson   NY  PG  77  35.4  28.7  10.3  21.4  47.9  2.7   
4             Kevin Durant  PHX  PF  75  37.2  27.1  10.0  19.1  52.3  2.2   

   ...   FT%   REB  AST  STL  BLK   TO  DD2  TD3  year    salary  
0  ...  78.6   9.2  9.8  1.4  0.5  4.0   49   21  2023  40064220  
1  ...  65.7  11.5  6.5  1.2  1.1  3.4   57   10  2023  45640084  
2  ...  87.4   5.5  6.2  2.0  0.9  2.2    8    0  2023  3338

In [2]:
# Agrégation simple : Moyenne par année
if 'year' in df.columns:
    result_simple = df.groupby('year')[['PTS', 'REB']].mean()
    print("Moyennes par année :")
    print(result_simple)
else:
    print("La colonne 'year' n'existe pas. Vérifiez le nom (ex: 'Season' ?)")

# --- RÉPONSE QUESTION 1 ---
# Que représente chaque ligne de result_simple ?
# R: Chaque ligne représente une année (saison). 
# Les valeurs sont la moyenne des points et des rebonds de TOUS les joueurs 
# ayant joué cette année-là. C'est la performance du "joueur moyen" sur la saison.

Moyennes par année :
            PTS       REB
year                     
2001   8.282324  3.762470
2002   9.531900  4.294624
2003   9.966854  4.539326
2004  10.263866  4.457983
2005   8.415691  3.656674
2006   8.510321  3.617431
2007   8.428146  3.635240
2008   8.567202  3.636468
2009   8.653118  3.683603
2010   8.389450  3.623853
2011  10.510460  4.494979
2012   8.293176  3.618588
2013   8.831084  3.779759
2014   8.483556  3.667333
2015   8.571047  3.704900
2016   8.642333  3.628294
2017   9.365000  3.809750
2018   9.447380  3.943450
2019  10.085926  4.149383
2020   9.888865  3.972489
2021   9.764026  3.958458
2022   9.846822  3.793856
2023  10.085517  3.911034


In [3]:
# Définir les fonctions d'agrégation
aggregations = {
    'PTS': ['sum', 'mean', 'max'],           # Total, moyenne, record de points
    'REB': ['sum', 'mean', 'std'],           # Total, moyenne, volatilité
    'salary': ['sum', lambda x: x.quantile(0.9)] # Masse salariale et top 10% salaire
}

# Appliquer le groupby
# Note : On vérifie que les colonnes existent avant
cols_presentes = [c for c in aggregations.keys() if c in df.columns]
agg_filtered = {k: v for k, v in aggregations.items() if k in cols_presentes}

result_adv = df.groupby('year').agg(agg_filtered)
print("Agrégation avancée :")
print(result_adv)

# --- RÉPONSE QUESTION 2 ---
# Expliquer ce que calcule chaque colonne :
# - PTS sum : Le nombre total de points marqués dans la ligue cette année-là.
# - PTS mean : La moyenne de points par joueur.
# - PTS max : Le record de points marqués par un seul joueur cette année-là.
# - REB std : L'écart-type (standard deviation), montre si les rebonds sont répartis
#   uniformément ou s'il y a de gros écarts entre les joueurs.
# - salary <lambda> : Le "90ème percentile", c'est-à-dire le seuil de salaire 
#   au-dessus duquel se trouvent les 10% des joueurs les mieux payés.

# --- RÉPONSE QUESTION 3 ---
# Pourquoi une fonction lambda ?
# R: Les chaînes de caractères ('mean', 'sum') ne prennent pas d'arguments.
# Pour utiliser une fonction qui a besoin d'un paramètre (comme quantile(0.9)), 
# on doit utiliser une lambda ou définir une fonction personnalisée.

Agrégation avancée :
         PTS                      REB                          salary  \
         sum       mean   max     sum      mean       std         sum   
year                                                                    
2001  3420.6   8.282324  31.4  1553.9  3.762470  2.483855  1414961000   
2002  2659.4   9.531900  32.1  1198.2  4.294624  2.515321  1335412000   
2003  1774.1   9.966854  28.0   808.0  4.539326  2.621709  1045535000   
2004  1221.4  10.263866  30.7   530.5  4.457983  2.447638   819341000   
2005  3593.5   8.415691  35.4  1561.4  3.656674  2.404070  1718426001   
2006  3710.5   8.510321  31.6  1577.2  3.617431  2.471338  1732391321   
2007  3683.1   8.428146  30.0  1588.6  3.635240  2.535255  1829968180   
2008  3735.3   8.567202  30.2  1585.5  3.636468  2.481578  2008524135   
2009  3746.8   8.653118  30.1  1595.0  3.683603  2.509645  1928603544   
2010  3657.8   8.389450  27.7  1580.0  3.623853  2.480977  1907104608   
2011  2512.0  10.510460  28.0 

In [4]:
# Group by team et position
# On s'assure que les colonnes existent (parfois POS est écrit Pos, team est Tm)
cols_group = []
if 'team' in df.columns: cols_group.append('team')
elif 'Tm' in df.columns: cols_group.append('Tm')

if 'POS' in df.columns: cols_group.append('POS')
elif 'Pos' in df.columns: cols_group.append('Pos')

if len(cols_group) == 2:
    team_pos_agg = df.groupby(cols_group).agg({
        'PTS': ['mean', 'max'],
        'AST': 'mean',
        'REB': 'mean'
    })
    print("Agrégation par Équipe et Position (Extrait) :")
    print(team_pos_agg.head(10))
else:
    print("Colonnes équipe ou position introuvables.")

# --- RÉPONSE QUESTION 4 ---
# Quelle est la différence avec l'agrégation par année ?
# R: L'index est "Hiérarchique" (MultiIndex). 
# Au lieu d'avoir une ligne par année, nous avons une ligne pour chaque combinaison
# unique d'équipe et de poste (ex: Les Meneurs (PG) des Lakers). 
# C'est une analyse beaucoup plus fine (granulaire).

Agrégation par Équipe et Position (Extrait) :
                   PTS             AST       REB
                  mean   max      mean      mean
team    POS                                     
ATL     C     7.766667  18.6  1.060784  5.649020
        F     7.496429  14.8  1.075000  3.985714
        G     9.864103  25.0  2.758974  2.569231
        PF    9.585000  21.6  1.458333  5.076667
        PG   12.645161  29.6  4.864516  2.251613
        SF    9.232258  20.8  1.500000  3.496774
        SG    7.980000  22.5  1.902500  2.417500
ATL/BOS C     2.300000   2.3  0.400000  1.000000
        PF   19.100000  19.1  3.400000  9.000000
ATL/CHI PF    5.600000   5.6  0.600000  1.900000


In [5]:
print("--- Exploration ---")

# 1. Identifier l'année où les joueurs ont marqué le plus de points (au total)
# On regarde result_adv, colonne PTS -> sum
if 'PTS' in result_adv.columns:
    annee_record = result_adv['PTS']['sum'].idxmax()
    max_pts = result_adv['PTS']['sum'].max()
    print(f"Année avec le plus de points marqués : {annee_record} ({max_pts:,.0f} points)")

# 2. Identifier l'équipe et la position ayant la meilleure moyenne de passes (AST)
# On utilise team_pos_agg
if 'AST' in team_pos_agg.columns:
    # idxmax renvoie le tuple (Equipe, Position)
    meilleur_duo = team_pos_agg['AST']['mean'].idxmax()
    meilleure_moyenne = team_pos_agg['AST']['mean'].max()
    print(f"Meilleure moyenne de passes : {meilleur_duo} avec {meilleure_moyenne:.2f} passes/match")
    
    # Pour voir le top 5
    print("\nTop 5 des positions par passes décisives :")
    print(team_pos_agg['AST']['mean'].sort_values(ascending=False).head(5))

--- Exploration ---
Année avec le plus de points marqués : 2022 (4,648 points)
Meilleure moyenne de passes : ('BKN/HOU', 'SG') avec 10.80 passes/match

Top 5 des positions par passes décisives :
team     POS
BKN/HOU  SG     10.8
UTAH/NJ  PG     10.3
NJ/DAL   PG     10.1
SEA/MIL  PG      8.3
NY/DEN   G       8.3
Name: mean, dtype: float64
